In [1]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [3]:

!kaggle datasets download -d hari31416/food-101

Dataset URL: https://www.kaggle.com/datasets/hari31416/food-101
License(s): other
 99% 4.64G/4.66G [00:56<00:00, 42.2MB/s]
100% 4.66G/4.66G [00:56<00:00, 89.3MB/s]


In [4]:
!unzip -o -q food-101.zip -d ./food-101




In [5]:
!ls ./food-101


food-101


In [6]:
!ls ./food-101/food-101


train  validation


In [7]:
import tensorflow as tf

train_dir = "./food-101/food-101/train"
val_dir = "./food-101/food-101/validation"

In [8]:
import os
import pandas as pd

train_dir = './food-101/food-101/train'

data = []

for class_name in sorted(os.listdir(train_dir)):
    class_path = os.path.join(train_dir, class_name)
    for fname in os.listdir(class_path):
        img_path = os.path.join(class_path, fname)
        data.append({'class': class_name, 'file': img_path})

df = pd.DataFrame(data)
df.head(10)


,class,file
0,apple_pie,./food-101/food-101/train/apple_pie/2740848.jpg
1,apple_pie,./food-101/food-101/train/apple_pie/3252424.jpg
2,apple_pie,./food-101/food-101/train/apple_pie/250066.jpg
3,apple_pie,./food-101/food-101/train/apple_pie/939246.jpg
4,apple_pie,./food-101/food-101/train/apple_pie/490047.jpg
5,apple_pie,./food-101/food-101/train/apple_pie/947689.jpg
6,apple_pie,./food-101/food-101/train/apple_pie/3589097.jpg
7,apple_pie,./food-101/food-101/train/apple_pie/1103795.jpg
8,apple_pie,./food-101/food-101/train/apple_pie/959678.jpg
9,apple_pie,./food-101/food-101/train/apple_pie/3670966.jpg


In [9]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(224, 224),
    batch_size=32
)

Found 95950 files belonging to 101 classes.
Found 5050 files belonging to 101 classes.


NOW MODEL BUILDING


In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models


train_ds = tf.keras.utils.image_dataset_from_directory(
    "./food-101/food-101/train",
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "./food-101/food-101/validation",
    image_size=(224, 224),
    batch_size=32
)


base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # freeze the base

model = models.Sequential([
    tf.keras.layers.Rescaling(1./255),
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(101, activation='softmax')  # 101 food classes
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Found 95950 files belonging to 101 classes.
Found 5050 files belonging to 101 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ ?                      │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
 history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)


Epoch 1/5
2443/2999 ━━━━━━━━━━━━━━━━━━━━ 13:20 1s/step - accuracy: 0.3971 - loss: 2.5447

In [ ]:
calorie_map = {
    'apple_pie': 296,
    'baby_back_ribs': 400,
    'baklava': 334,
    'beef_carpaccio': 150,
    'beet_salad': 90,
    'beignets': 289,
    'bibimbap': 560,
    'bread_pudding': 300,
    'breakfast_burrito': 305,
    'bruschetta': 170,
    'caesar_salad': 180,
    'cheesecake': 320,
    'churros': 280,
    'club_sandwich': 350,
    'cup_cakes': 210,
    'donuts': 270,
    'french_fries': 312,
    'frozen_yogurt': 120,
    'garlic_bread': 350,
    'hamburger': 290,
    # Add more if needed
}


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

class_names = train_ds.class_names  # 101 class names

def predict_image(img_path, model):
    img = tf.keras.utils.load_img(img_path, target_size=(224, 224))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # batch dim
    img_array = img_array / 255.0  # normalize

    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class = class_names[predicted_class_index]
    confidence = round(100 * np.max(predictions[0]), 2)

    calories = calorie_map.get(predicted_class, "Unknown")

    plt.imshow(img)
    plt.axis('off')
    plt.title(f"{predicted_class} ({confidence}%)\nCalories: {calories}")
    plt.show()

    return predicted_class, confidence, calories


In [ ]:
predict_image('/content//content/food-101/food-101/validation/beef_carpaccio/1099932.jpg', model)
